In [9]:
KITTI_DATA_DIR = 'G:/내 드라이브/data/kitti'


In [4]:
from typing import List, Iterable, Mapping, Dict, Any, Optional, IO
from abc import ABC, abstractmethod

In [ ]:
class MOTSequence(ABC):
    def __init__(self, det_source: str, seg_source: str, split_dir: str, name: str, frame_names: Iterable[str]):
        self.det_source = det_source
        self.seg_source = seg_source
        self.split_dir = split_dir
        self.name = name
        self.frame_names = frame_names
        
        # Image size for each camera - needed for 3D->2D projections. The dict is set in dataset-specific classes
        self.img_shape_per_cam: Dict[str, Any] = {}

        # Detections 3D {frame_name: [bboxes_3d]}
        self.dets_3d_per_frame: Dict[str, List[bbox.Bbox3d]] = {}

        # Detections 2D {frame_name: {cam_name: [bboxes_3d]}}
        self.dets_2d_multicam_per_frame: Dict[str, Dict[str, List[Detection2D]]] = {}

        # need to set its Transformation object and img_shape_per_cam in subclasses
        self.mot = TrackManager(self.cameras, self.classes_to_track)

        det_seg_source_folder_name = f'{self.det_source}_{self.seg_source}'
        self.work_split_input_dir = os.path.join(self.split_dir, det_seg_source_folder_name)
        self.tracking_res_dir = os.path.join(self.work_split_input_dir, 'tracking')

In [8]:
class MOTDataset(ABC):
    def __init__(self, work_dir: str, det_source: str, seg_source: str):
        """ Initialize the general dataset-level object

        :param work_dir: path to workspace output directory
        :param det_source: source of 3D detections
        :param seg_source: source of 2D detections
        """
        self.work_dir = work_dir
        self.det_source = det_source  # see dataset specific classes e.g. mot_kitti
        self.seg_source = seg_source  # see dataset specific classes e.g. mot_kitti
        self.splits: Set[str] = set()

    def assert_split_exists(self, split: str) -> None:
        assert split in self.splits, f"There is no split {split}"

    def assert_sequence_in_split_exists(self, split: str, sequence_name: str) -> None:
        self.assert_split_exists(split)
        assert sequence_name in self.sequence_names(split), f"There is no sequence {sequence_name} in split {split}"

    @abstractmethod
    def sequence_names(self, split: str) -> list[str]:
        """ Return list of sequences in the split """
        pass

    @abstractmethod
    def get_sequence(self, split: str, sequence_name: str):
        """ Return a sequence object by split-name combo"""
        pass

    @abstractmethod
    def save_all_mot_results(self, folder_name: str) -> None: pass

In [13]:
import os
import glob

In [ ]:
class MOTDatasetKITTI(MOTDataset):
    FOCAL = 721.537700
    CU = 609.559300
    CV = 172.854000
    BASELINE = 0.532719
    CAMERA_PARAMS = [FOCAL, CU, CV, BASELINE]

    def __init__(self, work_dir, det_source: str, seg_source: str):
        super().__init__(work_dir, det_source, seg_source)
        self.splits: Set[str] = {"training", "testing"}
        self.split_sequence_frame_names_map: Dict[str, Dict[str, List[str]]] = {sp: {} for sp in self.splits}

        for split in self.splits:
            seq_dir = os.path.join(KITTI_DATA_DIR, split, 'image_02')
            if not os.path.isdir(seq_dir):
                raise NotADirectoryError(seq_dir)

            # Parse sequences
            for sequence in sorted(os.listdir(seq_dir)):
                img_dir = os.path.join(seq_dir, sequence)
                if os.path.isdir(img_dir):
                    images = glob.glob(os.path.join(img_dir, '*.png'))
                    self.split_sequence_frame_names_map[split][sequence] = [os.path.splitext(os.path.basename(image))[0]
                                                                            for image in sorted(images)]

    def sequence_names(self, split: str) -> List[str]:
        self.assert_split_exists(split)
        return list(self.split_sequence_frame_names_map[split].keys())

    def get_sequence(self, split: str, sequence_name: str):
        self.assert_sequence_in_split_exists(split, sequence_name)
        split_dir = os.path.join(self.work_dir, split)
        return MOTSequenceKITTI(self.det_source, self.seg_source, split_dir, split, sequence_name,
                                self.split_sequence_frame_names_map[split][sequence_name])

    def save_all_mot_results(self, folder_name: str) -> None:
        """ KITTI saves results per-sequence, so this method does not apply here """
        pass